<center><h1>Data Generation Experiments</h1></center>

<img src="logo_autodata.png" alt="drawing" width="200"/>

### Datasets

* Description of the datasets

### Metrics

* Mainly "adversarial scores" with different discriminant models.
* Do the different models agree on the discrepancy between distributions?
* Can we rely on this kind of metric?

### Generators

* Which generators seems to have the best guarantees of similarity and privacy?

In [6]:
# ignore warning for a more convenient display
import warnings
warnings.filterwarnings('ignore')

In [7]:
import autopandas as apd
import numpy as np
from sklearn.neural_network import MLPClassifier
from IPython.display import display
%reload_ext autoreload
%autoreload 2

In [8]:
def load_dataset(input_file, test_file=None):
    if test_file is None:
        data = apd.read_csv(input_file)
    else:
        train = apd.read_csv(input_file)
        test = apd.read_csv(test_file)
        data = apd.from_train_test(train, test)
    # processing
    data = data.imputation()
    data = data.encoding()
    data = data.normalization()
    # TMP reduce size
    #data = data[:min(len(data), 200)]
    return data

In [9]:
def describe_dataset(data):
    # TODO
    # read data
    pass
    # cat/num
    # descriptors
    # PCA, TSNE, LDA, y dist

In [10]:
def run_experiment(datasets_list, generators_list):
    # Generate data and compute metrics
    for dataset in datasets_list:
        data = load_dataset(dataset)
        print('\n')
        print(dataset)
        print('=================================================')
        describe_dataset(data)
        
        # to store results
        names = ['Generator']
        nnaas = ['NN AA']
        discrepancies = ['NN AA (sklearn)']
        discriminants1 = ['LogReg AA']
        discriminants2 = ['MLP AA']
        
        for generator in generators_list:
            # init generator
            if generator is None:
                generator = apd.generators.Copycat
            generator = generator()
            
            names.append(generator.__class__.__name__)
            
            # training
            generator.fit(data)
            gen_data = generator.sample(n=data.shape[0])
            
            # TODO train/test
            # metrics
            nnaas.append(data.distance(gen_data, method='nnaa'))
            discrepancies.append(data.distance(gen_data))
            discriminants1.append(data.distance(gen_data, method='discriminant'))
            mlp = MLPClassifier()
            discriminants2.append(data.distance(gen_data, method='discriminant', model=mlp))
            
            # task score
            # TODO
            
            # plots
            # overlaying PCA and all
            #data.compare_marginals(gen_data, method='all')
            
        # Store results into a table
        results = np.array([names, nnaas, discrepancies, discriminants1, discriminants2]).T
        results = apd.AutoData(results[1:,:], columns=results[0,:])
        display(results)

In [12]:
path = 'autopandas/data/'
datasets = [path+x for x in ['iris.csv', 'wine.csv']] #, 'diabetes.csv', 'seeds.csv', 'adult.csv',
                             #'mushrooms.csv', 'wine.csv', 'squares.csv']]
# train/test boston, titanic
generators = [apd.generators.Copycat,
              apd.generators.KDE,
              apd.generators.ANM,
              apd.generators.Copula,
              apd.generators.GMM] # VAE, KDE, +

run_experiment(datasets, generators)



autopandas/data/iris.csv


,Generator,NN AA,NN AA (sklearn),LogReg AA,MLP AA
0,Copycat,0.0,0,0.5,0.5
1,KDE,0.5566666666666666,0.6866666666666665,0.5555555555555556,0.9222222222222223
2,ANM,0.5700000000000001,0.07333333333333325,0.5222222222222223,0.4888888888888889
3,Copula,0.6166666666666667,0.6000000000000001,0.4777777777777778,0.8333333333333334
4,GMM,0.7,0.5133333333333334,0.5333333333333333,0.6222222222222222
